In [32]:
import json
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from scipy.spatial import distance
from multiprocessing import Pool
from ipywidgets import interactive, widgets

In [33]:
ox_dir = 'data/oxford_robotcar'
datasets = ['sd2wd', 'wd2wn']

def select_dataset(d=datasets):
    return d

dataset_selected = interactive(select_dataset, dataset_selected=datasets)
display(dataset_selected)

interactive(children=(Dropdown(description='d', options=('sd2wd', 'wd2wn'), value='sd2wd'), Output()), _dom_cl…

In [55]:
dataset = dataset_selected.result
dataset

'wd2wn'

In [56]:
if dataset == 'sd2wd':
    route_1 = '2015-05-19-14-06-38'  # summer (day)
    route_2 = '2014-12-09-13-21-02'  # winter (day)
elif dataset == 'wd2wn':
    route_1 = '2014-12-09-13-21-02'  # winter (day)
    route_2 = '2014-12-10-18-10-50'  # winter (night)

## Load pose files

In [57]:
db_pose_file = os.path.join(ox_dir, route_1, "gps", f"poses.csv")
db_pose_df = pd.read_csv(db_pose_file, comment="#")
db_pose_df

,timestamp,ins_status,latitude,longitude,altitude,northing,easting,down,utm_zone,velocity_north,velocity_east,velocity_down,roll,pitch,yaw,imgs
0,1.418132e+09,INS_SOLUTION_GOOD,51.760857,-1.261838,107.090348,5.735870e+06,619956.293047,-107.090348,30U,-0.041037,-0.021394,-0.016622,4.667142,0.163579,203.716078,"{""stereoCentre"":""1418132416737115.jpg""}"
1,1.418132e+09,INS_SOLUTION_GOOD,51.760857,-1.261838,107.090382,5.735870e+06,619956.293004,-107.090382,30U,-0.040689,-0.021555,-0.016628,4.667242,0.163363,203.716131,"{""stereoCentre"":""1418132416799606.jpg""}"
2,1.418132e+09,INS_SOLUTION_GOOD,51.760857,-1.261838,107.090416,5.735870e+06,619956.292960,-107.090416,30U,-0.040342,-0.021716,-0.016635,4.667341,0.163147,203.716184,"{""stereoCentre"":""1418132416862098.jpg""}"
3,1.418132e+09,INS_SOLUTION_GOOD,51.760857,-1.261838,107.090450,5.735870e+06,619956.292917,-107.090450,30U,-0.039994,-0.021877,-0.016641,4.667440,0.162930,203.716236,"{""stereoCentre"":""1418132416924588.jpg""}"
4,1.418132e+09,INS_SOLUTION_GOOD,51.760857,-1.261838,107.090484,5.735870e+06,619956.292874,-107.090484,30U,-0.039647,-0.022039,-0.016648,4.667540,0.162714,203.716289,"{""stereoCentre"":""1418132416987080.jpg""}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34123,1.418135e+09,INS_SOLUTION_GOOD,51.760857,-1.261756,108.302249,5.735870e+06,619961.947757,-108.302249,30U,0.012106,-0.056558,-0.001496,0.340942,-0.262347,293.019973,"{""stereoCentre"":""1418134776790489.jpg""}"
34124,1.418135e+09,INS_SOLUTION_GOOD,51.760857,-1.261756,108.302306,5.735870e+06,619961.944299,-108.302306,30U,0.012250,-0.053507,-0.000544,0.338102,-0.263818,293.020292,"{""stereoCentre"":""1418134776852978.jpg""}"
34125,1.418135e+09,INS_SOLUTION_GOOD,51.760857,-1.261756,108.302380,5.735870e+06,619961.941028,-108.302380,30U,0.011096,-0.051250,-0.002252,0.337151,-0.265226,293.019971,"{""stereoCentre"":""1418134776915469.jpg""}"
34126,1.418135e+09,INS_SOLUTION_GOOD,51.760857,-1.261756,108.302600,5.735870e+06,619961.937775,-108.302600,30U,0.009921,-0.052876,-0.004582,0.340401,-0.267837,293.019599,"{""stereoCentre"":""1418134776977960.jpg""}"


In [58]:
q_pose_file = os.path.join(ox_dir, route_2, "gps", f"poses.csv")
q_pose_df = pd.read_csv(q_pose_file, comment="#")

## Filter close poses

Filter out poses that are too close to each other.

In [59]:
def filter_close_poses(pose_df, dist_threshold):
    pose_df = pose_df.copy()
    ix_rejected = set()
    for i in tqdm(range(0, len(pose_df))):
        if i in ix_rejected:
            continue
        q = pose_df.iloc[i]
        q_utm = q[['northing', 'easting']].to_numpy(dtype=np.float64)
        o = pose_df.iloc[i+1:]
        o_utms = o[['northing', 'easting']].to_numpy(dtype=np.float64)
        dists = distance.cdist([q_utm], o_utms).flatten()
        ix_close = np.where(dists < dist_threshold)[0]
        ix_close = ix_close + i + 1
        ix_rejected.update(ix_close)

    return pose_df.drop(ix_rejected)

q_pose_df = filter_close_poses(q_pose_df, dist_threshold=4)
print(f"{len(q_pose_df)} query poses after filtering")


100%|██████████| 38315/38315 [00:01<00:00, 24308.69it/s]

1925 query poses after filtering


## Find closest match

Find closest matching pair for each query pose

In [60]:
t_dist = 30
# t_angle = 30
pairs = []

def get_closest_pose(q_i):
    q = q_pose_df.iloc[q_i]
    q_utm = q[['northing', 'easting']].to_numpy(dtype=np.float64)
    q_yaw = q['yaw']

    # angles = np.abs(q_yaw - db_pose_df['yaw'])
    # valid_angles = angles <= t_angle
    
    db_utms = db_pose_df[['northing', 'easting']].to_numpy(dtype=np.float64)
    dists = distance.cdist([q_utm], db_utms).flatten()
    valid_dists = dists <= t_dist
    valid = valid_dists
    # valid = valid_angles & valid_dists

    if any(valid):
        # loss = (0.5 * dists) + (angles)
        loss = dists
        db_i = loss.argmin()
        return (q_i, db_i)

    return None

q_len = 5
# q_len = len(q_pose_df)
for q_i in tqdm(range(q_len), total=q_len):
    pairs_ret = get_closest_pose(q_i)
    if pairs_ret is not None:
        pairs.append(pairs_ret)

pairs

100%|██████████| 5/5 [00:00<00:00, 224.71it/s]


[(0, 34121), (1, 34121), (2, 34121), (3, 34127), (4, 61)]

In [61]:
threads = 8
q_len = len(q_pose_df)

with Pool(threads) as p:
    pairs_ret_list = list(tqdm(p.imap(get_closest_pose, range(q_len)), total=q_len))

100%|██████████| 1925/1925 [00:00<00:00, 6159.17it/s]


## Save to disk

Filter None matches and save to disk.

In [62]:
pairs = []
for pairs_ret in pairs_ret_list:
    if pairs_ret is not None:
        pairs.append(pairs_ret)


In [63]:
pairs = np.array(pairs)
q_selected = pairs[:, 0]
db_selected = pairs[:, 1]

output_dir = os.path.join(ox_dir, f"{dataset}")
os.makedirs(output_dir, exist_ok=True)
q_selected_file = os.path.join(output_dir, f"q_selected.csv")
db_selected_file = os.path.join(output_dir, f"db_selected.csv")

with open(db_selected_file, 'w') as f:
    print(f"# Sampled from", file=f)
    print(f"# Dataset: Oxford Robotcar", file=f)
    print(f"# Route: {route_1}", file=f)
db_pose_df.iloc[db_selected].to_csv(db_selected_file, index=False, mode='a')

with open(q_selected_file, 'w') as f:
    print(f"# Sampled from", file=f)
    print(f"# Dataset: Oxford Robotcar", file=f)
    print(f"# Route: {route_2}", file=f)
q_pose_df.iloc[q_selected].to_csv(q_selected_file, index=False, mode='a')